In [1]:
# %%
import cupy as cp
import numpy as np
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu.set import params, argv, set_ptrs
import h5py
print('My rank is ', define.rank)
# %%
gauge_filename = f"quda_wilson-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.bin"
print("Gauge filename:", gauge_filename)
gauge = cp.fromfile(gauge_filename, dtype=cp.complex64,
                    count=params[define._LAT_XYZT_]*define._LAT_DCC_)
gauge = io.gauge2ccdptzyx(gauge, params)
print("Gauge data:", gauge.data)
print("Gauge shape:", gauge.shape)
params[define._NODE_RANK_] = 5
params[define._NODE_SIZE_] = 8
params[define._GRID_T_] = 2
params[define._GRID_Z_] = 1
params[define._GRID_Y_] = 1
params[define._GRID_X_] = 4
print("Params:", params)


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

In [ ]:
print("params[define._NODE_RANK_]", params[define._NODE_RANK_])
print("params[define._NODE_SIZE_]", params[define._NODE_SIZE_])
_ = io.xxxtzyx2grid_xxxtzyx(gauge, params)

io.grid_xxxtzyx2hdf5_xxxtzyx(_, params)

params[define._NODE_RANK_] 5
params[define._NODE_SIZE_] 8
Input Array Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Grid Index T: 1, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 1
Dest Shape: (3, 3, 4, 2, 16, 32, 32, 4)
Input Array Shape: (3, 3, 4, 2, 16, 32, 32, 4)
Grid Index T: 1, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 1
Grid Lat T: 16, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 4
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Rank 5: Data is saved to xxxtzyx.h5


In [10]:
with h5py.File("./_xxxtzyx.h5", 'r', driver='mpio', comm=define.comm) as f:
    all_dset = f['data']
    dest=cp.array(all_dset)
print(dest-gauge)
print(f"Norm of difference: {cp.linalg.norm(dest-gauge)}")

[[[[[[[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       ...
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]

      [[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       ...
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]

      [[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       ...
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
       [0.+0.j 0.+0.j 0.+0.j

In [5]:

# qcu.applyEndQcu(set_ptrs, params)